In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
following = pd.read_json('following.json')
notifications = pd.read_json('notifications.json')
posts = pd.read_json('posts.json')
users = pd.read_json('users.json')

In [3]:
#Dropping irrevelant columns and renaming some columns
#column are renamed based on context for easier computing

following.drop('status', axis=1, inplace=True)
following.rename(columns={'my_id': 'followed_id', 'follower_id': 'user_id'}, inplace=True)
following.head()

,followed_id,user_id
0,3,6
1,6,3
2,3,2
3,3,7
4,7,2


In [34]:
cols = ['name', 'username', 'email', 'image', 'provider', 'provider_id', 'password', 'remember_token', 'created_at', 'updated_at', 'short_bio']

users_df = users.drop(cols, axis=1)
users_df.rename(columns={'id': 'user_id'}, inplace=True)
users_df.head()

,user_id
0,1
1,2
2,3
3,4
4,5


In [61]:
col = ['slug', 'created_at', 'updated_at', 'image', 'action', 'status_id', 'post_id']

posts_df = posts.drop(col, axis=1)
posts_df.rename(columns={'id': 'post_id'}, inplace = True)
posts_df.head()

,post_id,user_id,title,content,tags
0,1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,None
1,2,1719,HTML BEGINS HERE,"Â I am on this journey with start.ng, and here...",Technology
2,4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,None
3,6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,None
4,7,167,Task 2,"Â A Summary on The â€œidongesit.htmlâ€� CV, It...",None


In [36]:
notif_col = ['id', 'parent_comment_id', 'status', 'type', 'created_at', 'updated_at']
notif_df = notifications.drop(notif_col, axis=1)
notif_df.head()

,post_id,comment,sender_id,user_id,action
0,NaN,None,4,7,Followed
1,NaN,None,1,4,Followed
2,NaN,None,3,45,Followed
3,NaN,None,1,4,Followed
4,278.0,Hey bro!,1,4,Commented


In [37]:
len(posts_df.post_id.unique()) #From analysis we have 876 unique posts
len(posts_df.user_id.unique()) #And from analysis only 358 unique users have posted a topic

308

In [38]:
len(notif_df.post_id.unique()) #From analysis we only have 20 unique posts with notifications(comment, like or love)
#also from the table, sub category action: 'Followed' has no post id

20

In [39]:
len(following.user_id.unique()) # 878 unique users that have one or more users
len(following.followed_id.unique()) # 980 unique users that have been followed by one or more users

980

In [40]:
len(users_df.user_id.unique())  #2293 unique users

2293

Two recommender models will be used, popularity based, for users who have not read any post at all or visiting the blog, content based to recommend similar topics to users reading a particular post.

POPULARITY BASED MODEL

In [41]:
# A new table that is an contains only topics in both notification and post table

new_table = posts_df[posts_df['post_id'].isin(notif_df['post_id'])]
new_table = new_table.merge(notif_df.drop('user_id', axis=1), on='post_id')

In [42]:
new_table

,post_id,user_id,title,content,tags,comment,sender_id,action
0,143,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",None,I have learnt how to publish an article and Ho...,2191,Commented
1,278,4,Good Afternoon People,Hey,None,Hey bro!,1,Commented
2,353,7,TWO YEARS IN THE TRENCHES BUILDING HOTELS.NG,Hotels.ng is about two years old now. In the f...,None,None,1,Love
3,355,7,HOW TO AVOID BEING A ONE-HIT WONDER,"Most people, at some point, have a hit. Some h...",None,None,1,Like
4,355,7,HOW TO AVOID BEING A ONE-HIT WONDER,"Most people, at some point, have a hit. Some h...",None,Nice,1,Replied
5,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,This is an interesting write-up. I have a ques...,3,Commented
6,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,A nice write up,4,Commented
7,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,yh,4,Replied
8,371,3,Lucid is back,We have taken time to work on a lot of issues ...,None,That's awesome,9,Commented
9,371,3,Lucid is back,We have taken time to work on a lot of issues ...,None,"wait, this works?",6,Commented


In [43]:
# looking at the table you can see some people that liked and commented on their post, example: user_id = 3
# The new_columns action_counts is to compare user_id and sender_id, for sender_id not equal to user_id, count is 1 else 0
# This is to seive out real actions from noise

new_table['action_counts'] = [1 if line[2] != line[7] else 0 for line in new_table.itertuples()]
new_table

,post_id,user_id,title,content,tags,comment,sender_id,action,action_counts
0,143,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",None,I have learnt how to publish an article and Ho...,2191,Commented,0
1,278,4,Good Afternoon People,Hey,None,Hey bro!,1,Commented,1
2,353,7,TWO YEARS IN THE TRENCHES BUILDING HOTELS.NG,Hotels.ng is about two years old now. In the f...,None,None,1,Love,1
3,355,7,HOW TO AVOID BEING A ONE-HIT WONDER,"Most people, at some point, have a hit. Some h...",None,None,1,Like,1
4,355,7,HOW TO AVOID BEING A ONE-HIT WONDER,"Most people, at some point, have a hit. Some h...",None,Nice,1,Replied,1
5,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,This is an interesting write-up. I have a ques...,3,Commented,1
6,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,A nice write up,4,Commented,1
7,360,7,EVALUATING STARTUP IDEAS,The only true currency we humans have is time....,None,yh,4,Replied,1
8,371,3,Lucid is back,We have taken time to work on a lot of issues ...,None,That's awesome,9,Commented,1
9,371,3,Lucid is back,We have taken time to work on a lot of issues ...,None,"wait, this works?",6,Commented,1


In [44]:
pop_articles = new_table.groupby(['title'])['action_counts'].sum().sort_values(ascending=False)
pop_articles.index.name = 'Title'
print("TOP 10 RECOMMENDATIONS BASED ON MOST POPULAR ARTICLES\n")
print(pop_articles.head(10))

TOP 10 RECOMMENDATIONS BASED ON MOST POPULAR ARTICLES

Title
with picture                                                    6
Lucid is back                                                   6
this is mine                                                    4
CEO -- Story so Far                                             3
Your face PHP script                                            3
EVALUATING STARTUP IDEAS                                        3
DON'T ASK WHY YOU WERE REJECTED                                 2
HOW TO AVOID BEING A ONE-HIT WONDER                             2
A CHINESE VILLAGER WHO SELLS MORE SOFTWARE DAILY THAN YOU DO    2
THE SUNFLOWER                                                   2
Name: action_counts, dtype: int64


Content Based Filtering Method


In [73]:
# Importing packages

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from html.parser import HTMLParser

In [74]:
#For data cleaning
 
class MLStripper(HTMLParser):
    '''inherting the html parser to remove all matching html tags'''
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    '''functions that instantiates the MLStripper class and passed the text ''' 
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def strip_line(x):
    '''removes next line, return, image links and extra spaces'''
    stri = re.sub("\r|\n|(\[\]\(.*\))|\s+", " ", x)
    return stri.strip()


In [75]:
# Cleaning data
posts_df['content'] = posts_df['content'].map(strip_tags)
posts_df['content'] = posts_df['content'].map(strip_line)
posts_df['content'] = posts_df['content'].map(lambda x: x.strip().lower())
posts_df['title'] = posts_df['title'].apply(lambda x: x.strip().lower())

# features that will be usd to find similarity between posts
features = ['title', 'content', 'tags']

# loop through each feature column and replace nonetypes with empty string
for feature in features:
    posts_df[feature] = posts_df[feature].fillna('')


In [76]:
posts_df

,post_id,user_id,title,content,tags,combined_features
0,1,2077,what i have learnt so far on html,i learnt how to use the table tag as i have us...,,what i have learnt so far on html i learnt how...
1,2,1719,html begins here,"â i am on this journey with start.ng, and here...",Technology,html begins here â i am on this journey with s...
2,4,1310,my laziness in the open,i have not been attending classes on the hng c...,,my laziness in the open i have not been attend...
3,6,1787,my task 2,my journey on **startng** pre-internship progr...,,my task 2 my journey on **startng** pre-intern...
4,7,167,task 2,"â a summary on the â€œidongesit.htmlâ€� cv, it...",,task 2 â a summary on the â€œidongesit.htmlâ€�...
...,...,...,...,...,...,...
871,1003,45,jbhjkhkbb,jkhbjilbjhb v hvhjbv h hbv jhkv,,jbhjkhkbb jkhbjilbjhb v hvhjbv h hbv jhkv
872,1004,3,testing tags and really long posts,testing tags and really long posts. loren ipsu...,"Politics, Sports",testing tags and really long posts testing tag...
873,1005,3,testing image,test if image works,,testing image test if image works
874,1006,3,test,,,test


In [77]:
def combine_features(row):
    ''' combine features together to find similarity '''
    try:
        return row['title']+" "+row['content']+" "+row['tags']
    except:
        print("Error", row)

# creating a combined features column that has applyed the combine_features function to each row 
posts_df['combined_features'] = posts_df.apply(combine_features, axis=1)

posts_df

,post_id,user_id,title,content,tags,combined_features
0,1,2077,what i have learnt so far on html,i learnt how to use the table tag as i have us...,,what i have learnt so far on html i learnt how...
1,2,1719,html begins here,"â i am on this journey with start.ng, and here...",Technology,html begins here â i am on this journey with s...
2,4,1310,my laziness in the open,i have not been attending classes on the hng c...,,my laziness in the open i have not been attend...
3,6,1787,my task 2,my journey on **startng** pre-internship progr...,,my task 2 my journey on **startng** pre-intern...
4,7,167,task 2,"â a summary on the â€œidongesit.htmlâ€� cv, it...",,task 2 â a summary on the â€œidongesit.htmlâ€�...
...,...,...,...,...,...,...
871,1003,45,jbhjkhkbb,jkhbjilbjhb v hvhjbv h hbv jhkv,,jbhjkhkbb jkhbjilbjhb v hvhjbv h hbv jhkv
872,1004,3,testing tags and really long posts,testing tags and really long posts. loren ipsu...,"Politics, Sports",testing tags and really long posts testing tag...
873,1005,3,testing image,test if image works,,testing image test if image works
874,1006,3,test,,,test


In [84]:
#Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(posts_df['combined_features'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

#Construct a reverse map of indices and post titles
indices = pd.Series(posts_df.index, index=posts_df['title']).drop_duplicates()


In [90]:
# function that takes in a post title as an input
# this is when a user clicks on an article

def get_recommendations(title, cosine_sim=cosine_sim):
  
  try:
    idx = indices[title.lower()]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar posts
    sim_scores = sim_scores[1:11]

    # Get the post indices
    post_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar posts
    print("RECOMMENDATIONS BASED ON YOUR READING PATTERN\n")
    return posts['title'].iloc[post_indices]
  
  except KeyError:
    print("We Have No Recommendations For You!")


In [92]:
#calling function for recommendation
get_recommendations('Introduction To HTML')

RECOMMENDATIONS BASED ON YOUR READING PATTERN



1                         HTML BEGINS HERE
745                              CSS Rules
746                              CSS Rules
749                              CSS Rules
750                              CSS Rules
751                              CSS Rules
814       What i learnt  so far about HTML
63     BABY STEPS TO BECOMING A PROGRAMMER
707                           what is HTML
708                           What is HTML
Name: title, dtype: object